In [29]:
from langchain_community.document_loaders import PyPDFLoader

# https://python.langchain.com/docs/integrations/document_loaders/pypdfloader/#add-a-custom-pages_delimiter-to-identify-where-are-ends-of-pages-in-single-mode
def pdf_loader(pdf_path: str):
    return PyPDFLoader(
        pdf_path,
        mode="single",
        pages_delimiter="\n-------END OF PAGE-------\n",
    )

dream_dictionary_docs = pdf_loader("assets/data/The_Dreamers_Dictionary.pdf").load()
jung_archetypes_docs = pdf_loader("assets/data/The_Archetypes_of_the_Collective_Unconscious_C.Jung.pdf").load()
jung_interpretations_docs = pdf_loader("assets/data/Symbols_and_the_Interpretation_of_Dreams_by_Carl_Jung.pdf").load()
personality_types_docs = pdf_loader("assets/data/Understanding Personality - The 12 Jungian Archetypes.pdf").load()
freud_interpretations_docs = pdf_loader("assets/data/The Interpretation of Dreams - Sigmund Freud (1900).pdf").load()

In [30]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,                # chunk size (characters)
    chunk_overlap=200,              # chunk overlap (characters)
    add_start_index=True,           # track index in original document
)
dream_dictionary_splits = text_splitter.split_documents(dream_dictionary_docs)
jung_archetypes_splits = text_splitter.split_documents(jung_archetypes_docs)
jung_interpretations_splits = text_splitter.split_documents(jung_interpretations_docs)
personality_types_splits = text_splitter.split_documents(personality_types_docs)
freud_interpretations_splits = text_splitter.split_documents(freud_interpretations_docs)

In [31]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings  # local
from langchain_core.vectorstores import InMemoryVectorStore

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={"device": "cuda"}, # use 'cpu' in case you don't have a gpu
)

# Create In-Memory Vector Stores
dream_dictionary_store = InMemoryVectorStore(embeddings)
jung_archetypes_store = InMemoryVectorStore(embeddings)
jung_interpretations_store = InMemoryVectorStore(embeddings)
personality_types_store = InMemoryVectorStore(embeddings)
freud_interpretations_store = InMemoryVectorStore(embeddings)

In [32]:
print("\nattempting embed...\n")
dream_dictionary_store.add_documents(documents=dream_dictionary_splits)
jung_archetypes_store.add_documents(documents=jung_archetypes_splits)
jung_interpretations_store.add_documents(documents=jung_interpretations_splits)
personality_types_store.add_documents(documents=personality_types_splits)
freud_interpretations_store.add_documents(documents=freud_interpretations_splits)
print("done")


attempting embed...

done


In [33]:
import pickle

with open(
    "assets/pickles/dream_dictionary_store.dat", mode="wb"
) as f_dream_dictionary_store, open(
    "assets/pickles/jung_archetypes_store.dat", mode="wb"
) as f_jung_archetypes_store, open(
    "assets/pickles/jung_interpretations_store.dat", mode="wb"
) as f_jung_interpretations_store, open(
    "assets/pickles/personality_types_store.dat", mode="wb"
) as f_personality_types_store, open(
    "assets/pickles/freud_interpretations_store.dat", mode="wb"
) as f_freud_interpretations_store:
    pickle.dump(freud_interpretations_store, f_freud_interpretations_store)
    pickle.dump(dream_dictionary_store, f_dream_dictionary_store)
    pickle.dump(jung_archetypes_store, f_jung_archetypes_store)
    pickle.dump(jung_interpretations_store, f_jung_interpretations_store)
    pickle.dump(personality_types_store, f_personality_types_store)
    print("done storing")

done storing


In [ ]:
import pickle

with open(
    "assets/pickles/dream_dictionary_store.dat", mode="rb"
) as f_dream_dictionary_store, open(
    "assets/pickles/jung_archetypes_store.dat", mode="rb"
) as f_jung_archetypes_store, open(
    "assets/pickles/jung_interpretations_store.dat", mode="rb"
) as f_jung_interpretations_store, open(
    "assets/pickles/personality_types_store.dat", mode="rb"
) as f_personality_types_store, open(
    "assets/pickles/freud_interpretations_store.dat", mode="rb"
) as f_freud_interpretations_store:
    freud_interpretations_store = pickle.load(f_freud_interpretations_store)
    dream_dictionary_store = pickle.load(f_dream_dictionary_store)
    jung_archetypes_store = pickle.load(f_jung_archetypes_store)
    jung_interpretations_store = pickle.load(f_jung_interpretations_store)
    personality_types_store = pickle.load(f_personality_types_store)
    print("done loading")

UnsupportedOperation: read

In [28]:
from typing import (
    Optional,
    List,
    Tuple,
)  # https://docs.python.org/3/library/typing.html
from pydantic import BaseModel, Field
from langchain_ollama import ChatOllama
from pydantic._internal._model_construction import ModelMetaclass

llm = ChatOllama(
    model="llama3.2:3b",
    temperature=0,
    # other params...
)
print(type(llm))


dream_text = "I had a dream where I was in a crowd of pigeons and I couldn't move forward, but I was getting late for my dance competition. Suddenly, I was on my mom's shoulders and the pigeons turned into humans."
# dream_text = "I'm not sure if this was a dream of a hallucination, but I could see myself falling into the abyss as I was thinking of my next meeting. The very next thing I knew, I woke up sudeenly on my toilet seat, when I clearly remember being home."


class dream_classification(BaseModel):
    """Classify the kind of prompt. Probabilities range from 0 to 1."""

    # archetype: str = Field(description="Jungian archetype.")
    dream: float = Field(description="probability of it being a dream. Values can range from 0 to 1")
    question: float = Field(description="probability of it being a question being asked. Values can range from 0 to 1")
    statement: float = Field(description="probability of it being a general statement. Values can range from 0 to 1")
    truth: float = Field(description="probability of it being a well known truth. Values can range from 0 to 1")
    greeting: float = Field(description="probability of it being a greeting. Values can range from 0 to 1")


class archetype_classification(BaseModel):
    """Dream Analysis."""

    # archetype: str = Field(description="Jungian archetype.")
    archetype: str = Field(
        description="Archetype based on context and dream. It should be one of Ruler, Creator/Artist, Sage, Innocent, Explorer, Rebel, Hero, Wizard, Jester, Everyman, Lover, Caregiver"
    )


# print(run(dream_text, prompt, llm, archetype_classification))
dream_dictionary_results = dream_dictionary_store.similarity_search(dream_text)
jung_archetypes_results = jung_archetypes_store.similarity_search(dream_text)
jung_interpretations_results = jung_interpretations_store.similarity_search(dream_text)
personality_types_results = personality_types_store.similarity_search(dream_text)


def parse(result):
    return "\n\n".join(doc.page_content for doc in result)


dream_dictionary_context = parse(dream_dictionary_results)
jung_archetypes_context = parse(jung_archetypes_results)
jung_interpretations_context = parse(jung_interpretations_results)
personality_types_context = parse(personality_types_results)


prompt = f"Dream: {dream_text}\n\nJungian context: {personality_types_results}"
answer = llm.with_structured_output(archetype_classification).invoke(prompt)

print(
    answer.model_dump_json(),
    llm.with_structured_output(dream_classification)
    .invoke(f"Dream: {dream_text}")
    .model_dump_json(),
)

<class 'langchain_ollama.chat_models.ChatOllama'>
{"archetype":"Everyman"} {"dream":0.0,"question":1.0,"statement":0.0,"truth":0.0,"greeting":0.0}


<hr>

> testing out huggingface embeddinngs

In [ ]:
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate

load_dotenv()

# https://python.langchain.com/docs/integrations/chat/huggingface/#huggingfacepipeline
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.2-3B-Instruct",
    task="text-generation",
    temperature=0,
)
llm = ChatHuggingFace(llm=llm)

dream_text = "I dreamt of kissing my girlfriend"

dream_dictionary_results = dream_dictionary_store.similarity_search(dream_text)
jung_archetypes_results = jung_archetypes_store.similarity_search(dream_text)
jung_interpretations_results = jung_interpretations_store.similarity_search(dream_text)

def parse(result):
    return "\n\n".join(doc.page_content for doc in result)

dream_dictionary_context = parse(dream_dictionary_results)
jung_archetypes_context = parse(jung_archetypes_results)
jung_interpretations_context = parse(jung_interpretations_results)



In [ ]:
from typing import List
from langchain_core.tools import tool


@tool
def validate_user(archetype: str, interpretation: str) -> bool:
    """Interpret the user's dream using the given context.

    Args:
        archetype (str): Jungian archetype.
        interpretation (str): Interpretation of the dream based on the Jungian archetype.
    """
    return True


llm = llm.bind_tools([validate_user])

result = llm.invoke(
    f"""{dream_text}\n\nJungian archetype context: {jung_archetypes_context}"""
)
result.tool_calls

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from pprint import pprint
prompt = ChatPromptTemplate.from_template(
    """Interpret the user's dream using the given context. Give me the result in a json format only, with the keys: ['archetype', 'interpretation']

    dream: {dream_text}
    Jungian archetype context: {jung_archetypes_context}
    """
)

dream_text = "I was kissing my wife"

chain = prompt | llm
answer = chain.invoke({"dream_text": dream_text, "jung_archetypes_context": jung_interpretations_context})

pprint(answer)

<hr>